### **Nombre:** Mafer Arias    **Curso:** GR1CC

# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [23]:
def busqueda_lineal(corpus, query):
    resultados = {}

    for archivo in corpus:
        with open(archivo, 'r', encoding='utf-8') as f:
            for numero_linea, linea in enumerate(f, start=1):
                if query in linea:  # Se respeta mayúsculas y minúsculas
                    if archivo not in resultados:
                        resultados[archivo] = []
                    resultados[archivo].append((numero_linea, linea.strip()))

    return resultados

if __name__ == "__main__":
    corpus = [
        '/kaggle/input/data-prueba/01_corpus_turismo.txt',
        '/kaggle/input/data-prueba/01_corpus_turismo_500.txt'
    ]

    query = input("Ingrese la palabra que desea buscar: ").strip()
    resultados = busqueda_lineal(corpus, query)

    if resultados:
        print(f"\nLa consulta '{query}' fue encontrada en:")
        for archivo, coincidencias in resultados.items():
            print(f"\nArchivo: {archivo}")
            for numero_linea, contenido in coincidencias:
                print(f"- Línea {numero_linea}: {contenido}")
    else:
        print(f"\nLa palabra '{query}' no fue encontrada en ningún documento.")


Ingrese la palabra que desea buscar:  cultura



La consulta 'cultura' fue encontrada en:

Archivo: /kaggle/input/data-prueba/01_corpus_turismo.txt
- Línea 6: Otavalo es famoso por su mercado indígena y su vibrante cultura artesanal.


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [25]:
import string

def leer_documento(ruta_archivo):
    palabras_documento = {}

    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        for numero_linea, linea in enumerate(archivo, start=1):
            for palabra in linea.split():
                palabra = palabra.strip(string.punctuation)
                if palabra:
                    if palabra not in palabras_documento:
                        palabras_documento[palabra] = set()
                    palabras_documento[palabra].add(numero_linea)

    return palabras_documento

def construir_indice(corpus):
    indice = {}

    for ruta in corpus:
        palabras_en_doc = leer_documento(ruta)
        for palabra, lineas in palabras_en_doc.items():
            if palabra not in indice:
                indice[palabra] = {}
            indice[palabra][ruta] = sorted(lineas)

    return indice

def buscar_palabra(palabra, indice):
    return indice.get(palabra, {})

def mostrar_resultados(resultado):
    if resultado:
        print("\nResultados de búsqueda:")
        for documento, lineas in resultado.items():
            print(f"- {documento}: líneas {lineas}")
    else:
        print("\nLa palabra no se encontró en ningún documento.")

if __name__ == "__main__":
    data_prueba = [
        '/kaggle/input/data-prueba/01_corpus_turismo.txt',
        '/kaggle/input/data-prueba/01_corpus_turismo_500.txt'
    ]

    print("Construyendo el índice invertido...")
    indice_invertido = construir_indice(data_prueba)

    print(f"\nSe indexaron {len(indice_invertido)} palabras únicas.")

    palabra_buscar = input("\nIngrese la palabra que desea buscar: ").strip()
    resultados = buscar_palabra(palabra_buscar, indice_invertido)

    mostrar_resultados(resultados)


Construyendo el índice invertido...

Se indexaron 176 palabras únicas.



Ingrese la palabra que desea buscar:  Quito



Resultados de búsqueda:
- /kaggle/input/data-prueba/01_corpus_turismo.txt: líneas [3]
- /kaggle/input/data-prueba/01_corpus_turismo_500.txt: líneas [4, 16, 17, 39, 43, 73, 75, 89, 104, 136, 201, 213, 238, 260, 291, 325, 355, 391, 469, 487]


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [26]:
import string
import time
import pandas as pd

def busqueda_lineal(palabra, archivos_corpus):
    resultados = {}
    for archivo in archivos_corpus:
        with open(archivo, 'r', encoding='utf-8') as f:
            for numero_linea, linea in enumerate(f, start=1):
                if palabra in linea: 
                    if archivo not in resultados:
                        resultados[archivo] = []
                    resultados[archivo].append((numero_linea, linea.strip()))
    return resultados

def leer_documento(ruta_archivo):
    palabras_documento = {}
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        for numero_linea, linea in enumerate(archivo, start=1):
            for palabra in linea.split():
                palabra = palabra.strip(string.punctuation)
                if palabra:
                    if palabra not in palabras_documento:
                        palabras_documento[palabra] = set()
                    palabras_documento[palabra].add(numero_linea)
    return palabras_documento

def construir_indice(corpus):
    indice = {}
    for ruta in corpus:
        palabras_en_doc = leer_documento(ruta)
        for palabra, lineas in palabras_en_doc.items():
            if palabra not in indice:
                indice[palabra] = {}
            indice[palabra][ruta] = sorted(lineas)
    return indice

def busqueda_indice(palabra, indice):
    return indice.get(palabra, {})

if __name__ == "__main__":
    palabras_a_buscar = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

    corpus_pequeño = ['/kaggle/input/data-prueba/01_corpus_turismo.txt']
    corpus_grande = ['/kaggle/input/data-prueba/01_corpus_turismo_500.txt']

    resultados_lineal = []
    resultados_indice = []

    for corpus_nombre, corpus in [("Corpus Pequeño", corpus_pequeño), ("Corpus Grande", corpus_grande)]:
        for palabra in palabras_a_buscar:
            inicio = time.time()
            _ = busqueda_lineal(palabra, corpus)
            fin = time.time()
            resultados_lineal.append({
                "Método": "Lineal",
                "Corpus": corpus_nombre,
                "Palabra": palabra,
                "Tiempo (segundos)": round(fin - inicio, 5)
            })

    for corpus_nombre, corpus in [("Corpus Pequeño", corpus_pequeño), ("Corpus Grande", corpus_grande)]:
        indice = construir_indice(corpus)
        for palabra in palabras_a_buscar:
            inicio = time.time()
            _ = busqueda_indice(palabra, indice)
            fin = time.time()
            resultados_indice.append({
                "Método": "Índice Invertido",
                "Corpus": corpus_nombre,
                "Palabra": palabra,
                "Tiempo (segundos)": round(fin - inicio, 5)
            })

    df_lineal = pd.DataFrame(resultados_lineal)
    df_indice = pd.DataFrame(resultados_indice)

    print("\nTabla de búsqueda Lineal:\n")
    print(df_lineal.to_string(index=False))

    print("\nTabla de búsqueda con Índice Invertido:\n")
    print(df_indice.to_string(index=False))



Tabla de búsqueda Lineal:

Método         Corpus   Palabra  Tiempo (segundos)
Lineal Corpus Pequeño     quito            0.00225
Lineal Corpus Pequeño montañita            0.00052
Lineal Corpus Pequeño   feriado            0.00053
Lineal Corpus Pequeño    playas            0.00055
Lineal Corpus Pequeño  aventura            0.00054
Lineal Corpus Pequeño galápagos            0.00051
Lineal  Corpus Grande     quito            0.00066
Lineal  Corpus Grande montañita            0.00064
Lineal  Corpus Grande   feriado            0.00061
Lineal  Corpus Grande    playas            0.00065
Lineal  Corpus Grande  aventura            0.00060
Lineal  Corpus Grande galápagos            0.00059

Tabla de búsqueda con Índice Invertido:

          Método         Corpus   Palabra  Tiempo (segundos)
Índice Invertido Corpus Pequeño     quito                0.0
Índice Invertido Corpus Pequeño montañita                0.0
Índice Invertido Corpus Pequeño   feriado                0.0
Índice Invertido Corpus

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [27]:
import string
import time

def leer_documento(ruta_archivo):
    palabras_documento = {}

    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        for numero_linea, linea in enumerate(archivo, start=1):
            for palabra in linea.split():
                palabra = palabra.strip(string.punctuation).lower()
                if palabra:
                    if palabra not in palabras_documento:
                        palabras_documento[palabra] = set()
                    palabras_documento[palabra].add(numero_linea)

    return palabras_documento

def construir_indice(corpus):
    indice = {}
    for ruta in corpus:
        palabras_en_doc = leer_documento(ruta)
        for palabra, lineas in palabras_en_doc.items():
            if palabra not in indice:
                indice[palabra] = {}
            indice[palabra][ruta] = sorted(lineas)
    return indice

def buscar_invertido(palabras, indice):
    resultados = {}
    for palabra in palabras:
        palabra = palabra.lower()
        encontrados = indice.get(palabra, {})
        for doc, lineas in encontrados.items():
            if doc not in resultados:
                resultados[doc] = set()
            resultados[doc].update(lineas)
    return resultados

def buscar_lineal(corpus, palabras):
    resultados = {}
    for archivo in corpus:
        with open(archivo, 'r', encoding='utf-8') as f:
            for numero_linea, linea in enumerate(f, start=1):
                linea_limpia = linea.lower()
                if all(palabra.lower() in linea_limpia for palabra in palabras):
                    if archivo not in resultados:
                        resultados[archivo] = []
                    resultados[archivo].append(numero_linea)
    return resultados

def mostrar_resultados(nombre_metodo, resultado, tiempo):
    print(f"\nMétodo: {nombre_metodo}")
    print(f"Tiempo de ejecución: {tiempo:.6f} segundos")
    if resultado:
        for doc, lineas in resultado.items():
            print(f"- {doc}: líneas {sorted(lineas)}")
    else:
        print("No se encontraron coincidencias.")

if __name__ == "__main__":
    data_prueba = [
        '/kaggle/input/data-prueba/01_corpus_turismo.txt',
        '/kaggle/input/data-prueba/01_corpus_turismo_500.txt'
    ]

    consulta = input("Ingrese una o más palabras separadas por espacio: ").strip().lower()
    palabras = consulta.split()
    tiempo_inicio = time.time()
    indice_invertido = construir_indice(data_prueba)
    tiempo_indice = time.time() - tiempo_inicio
    print(f"Índice creado en {tiempo_indice:.6f} segundos con {len(indice_invertido)} palabras.\n")

    inicio_inv = time.time()
    resultados_inv = buscar_invertido(palabras, indice_invertido)
    tiempo_inv = time.time() - inicio_inv

    inicio_lin = time.time()
    resultados_lin = buscar_lineal(data_prueba, palabras)
    tiempo_lin = time.time() - inicio_lin

    mostrar_resultados("Búsqueda con índice invertido", resultados_inv, tiempo_inv)
    mostrar_resultados("Búsqueda lineal", resultados_lin, tiempo_lin)

    # Speedup
    if tiempo_inv > 0:
        speedup = tiempo_lin / tiempo_inv
        print(f"\nSpeedup (lineal / índice): {speedup:.2f}x")
    else:
        print("\nNo se puede calcular speedup: tiempo de índice invertido es 0.")


Ingrese una o más palabras separadas por espacio:  playa y rio


Índice creado en 0.013210 segundos con 166 palabras.


Método: Búsqueda con índice invertido
Tiempo de ejecución: 0.000026 segundos
- /kaggle/input/data-prueba/01_corpus_turismo.txt: líneas [1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]
- /kaggle/input/data-prueba/01_corpus_turismo_500.txt: líneas [1, 3, 6, 7, 11, 13, 15, 18, 21, 26, 29, 34, 40, 46, 50, 55, 57, 58, 60, 65, 71, 74, 80, 86, 92, 93, 95, 101, 102, 105, 108, 111, 112, 114, 116, 118, 119, 121, 123, 129, 131, 134, 140, 141, 143, 144, 145, 147, 148, 149, 155, 158, 159, 161, 165, 166, 169, 174, 176, 182, 183, 184, 186, 191, 193, 197, 199, 204, 207, 208, 214, 215, 216, 218, 221, 222, 226, 228, 230, 234, 240, 241, 245, 247, 252, 253, 263, 269, 270, 279, 282, 283, 285, 286, 287, 290, 292, 296, 297, 301, 302, 303, 305, 320, 326, 334, 335, 336, 340, 342, 343, 344, 351, 357, 358, 363, 364, 366, 367, 369, 373, 376, 380, 382, 393, 395, 397, 401, 402, 404, 407, 411, 415, 418, 424, 425, 426, 427, 429, 431, 432, 433, 435, 436, 444, 445, 4